In [1]:
from __future__ import print_function

In [2]:
import urllib
import bz2
import os
import numpy as np
import torch
from torch import autograd
from torch import optim

%matplotlib inline
import matplotlib.pyplot as plt

In [194]:
device = torch.device('cpu')

In [195]:
BASE_URL = "http://grail.cs.washington.edu/projects/bal/data/ladybug/"
FILE_NAME = "problem-49-7776-pre.txt.bz2"
URL = BASE_URL + FILE_NAME

In [196]:
if not os.path.isfile(FILE_NAME):
    urllib.request.urlretrieve(URL, FILE_NAME)

In [197]:
def read_bal_data(file_name):
    with bz2.open(file_name, "rt") as file:
        n_cameras, n_points, n_observations = map(int, file.readline().split())
    
        camera_indices = np.empty(n_observations, dtype = int)
        point_indices = np.empty(n_observations, dtype = int)
        points_2d = torch.empty(n_observations, 2, device = device)

        for i in range(n_observations):
            camera_index, point_index, x, y = file.readline().split()
            camera_indices[i] = camera_index
            point_indices[i] = point_index
            points_2d[i] = torch.tensor([float(x), float(y)])

            camera_params = torch.empty(n_cameras*9, device = device)

        for i in range(n_cameras*9):
            camera_params[i] = float(file.readline())

        camera_params = camera_params.view(n_cameras, -1)

        points_3d = torch.empty(n_points*3, device = device)

        for i in range(n_points*3):
            points_3d[i] = float(file.readline())
        points_3d = points_3d.view(n_points, -1)
            
    return camera_params, points_3d, camera_indices, point_indices, points_2d

In [227]:
c_params, p3d, camera_indices, point_indices, points_2d = read_bal_data(FILE_NAME)

In [228]:
camera_indices = torch.tensor(camera_indices, device = device)
point_indices = torch.tensor(point_indices, device = device)

In [229]:
n_cameras = c_params.size()[0]
n_points = p3d.size()[0]

n = 9*n_cameras + 3*n_points
m = 2*points_2d.size()[0]

print("n_cameras: {}".format(n_cameras))
print("n_points: {}".format(n_points))
print("Total number of parameters: {}".format(n))
print("Total number of residuals: {}".format(m))

n_cameras: 49
n_points: 7776
Total number of parameters: 23769
Total number of residuals: 63686


In [230]:
c_params.requires_grad_(True)
p3d.requires_grad_(True)

tensor([[-0.6120,  0.5718, -1.8471],
        [ 1.7075,  0.9539, -6.8772],
        [-0.3734,  1.5359, -4.7824],
        ...,
        [-0.6642, -0.1351, -5.5425],
        [-0.8193,  0.0765, -4.5143],
        [-0.7480,  0.0371, -4.8132]], requires_grad=True)

In [231]:
def rotate(points, rot_vecs):
    
    theta = torch.norm(rot_vecs, dim = 1, keepdim=True)
    v = rot_vecs/theta
    v[v != v] = 0.
#     print(v.size(), points.size())
    
    dot = torch.sum(points*v, dim = 1, keepdim = True)
    
    cos_theta = torch.cos(theta)
    sin_theta = torch.sin(theta)
    
    ans = cos_theta*points + sin_theta*torch.cross(v, points) + dot*(1-cos_theta)*v
    
    return ans

In [222]:
def project(points, camera_params):
    R = torch.index_select(camera_params, 1, torch.tensor([0,1,2]))
    T = torch.index_select(camera_params, 1, torch.tensor([3,4,5]))
    # print(R.size(), T.size())
    points_proj = rotate(points, R)
    points_proj = points_proj + T
    denom = torch.index_select(points_proj,1,torch.tensor([2])).view(-1,1)
    points_proj_2 = -torch.index_select(points_proj,1,torch.tensor([0,1]))/denom
#     f = camera_params[:,6]
#     k1 = camera_params[:,7]
#     k2 = camera_params[:,8]
    f = torch.index_select(camera_params, 1, torch.tensor([6]))
    k1 = torch.index_select(camera_params, 1, torch.tensor([7]))
    k2 = torch.index_select(camera_params, 1, torch.tensor([8]))
    
    n = torch.sum(torch.mul(points_proj_2,points_proj_2), dim = 1)
    # print(f.size(), k1.size(), k2.size(), n.size())
    r = 1 + torch.mul(n,k1.view(-1)) + torch.mul(k2.view(-1),torch.mul(n,n))
    # print(r.size(), f.size(), points_proj.size())
    # print(torch.mul(r,f.view(f.numel())).size())
    points_proj_3 = points_proj_2*torch.mul(r,f.view(-1)).unsqueeze(1)
    return points_proj_3

In [223]:
def fun(camera_params, points_3d, n_cameras, n_points, camera_indices, point_indices, points_2d):
    points_3d_2 = torch.index_select(points_3d, 0, point_indices)
    camera_params_2 = torch.index_select(camera_params, 0, camera_indices)
    points_proj = project(points_3d_2, camera_params_2)
    ans = points_proj - points_2d
    return ans.view(-1)

def loss_fn(f):
    "Squared error loss"
    return f.pow(2).sum()

In [224]:
def train(optimizer, epochs, loss_fn, fun):
    for i in range(epochs):
        def closure():
            optimizer.zero_grad()
        
            f = fun(c_params, p3d, n_cameras, n_points, camera_indices, point_indices, points_2d)
            loss = loss_fn(f)
        
            print(i, " --> ", loss.item())
        
            loss.backward()
            
            return loss
        optimizer.step(closure)

In [232]:
optimizer = optim.LBFGS([c_params, p3d], lr = 1)

In [233]:
train(optimizer, 500, loss_fn, fun)

0  -->  1701824.875
0  -->  2323797.0
0  -->  1365863.5
0  -->  965764.4375
0  -->  829458.375
0  -->  693203.4375
0  -->  580876.75
0  -->  521650.0625
0  -->  451944.9375
0  -->  431620.15625
0  -->  389903.1875
0  -->  364085.96875
0  -->  346371.0625
0  -->  292677.5625
0  -->  278450.78125
0  -->  240760.15625
0  -->  223787.890625
0  -->  206785.046875
0  -->  188462.90625
0  -->  189943.484375
1  -->  168350.359375
1  -->  163659.4375
1  -->  155261.3125
1  -->  145422.796875
1  -->  139561.15625
1  -->  131814.828125
1  -->  127621.984375
1  -->  120427.265625
1  -->  117690.09375
1  -->  109772.6015625
1  -->  106819.984375
1  -->  103316.875
1  -->  100191.28125
1  -->  96795.5078125
1  -->  94397.59375
1  -->  92651.90625
1  -->  90387.4140625
1  -->  88083.5
1  -->  86821.296875
1  -->  84636.265625
2  -->  83124.546875
2  -->  81375.390625
2  -->  80104.25
2  -->  78970.5546875
2  -->  77838.53125
2  -->  76074.953125
2  -->  75791.546875
2  -->  74464.25
2  -->  73932.273

18  -->  44475.1796875
18  -->  44467.734375
18  -->  44462.359375
18  -->  44456.70703125
18  -->  44449.94921875
18  -->  44441.89453125
18  -->  44435.4609375
19  -->  44430.6171875
19  -->  44421.171875
19  -->  44417.16015625
19  -->  44411.640625
19  -->  44402.4609375
19  -->  44402.265625
19  -->  44385.6640625
19  -->  44381.140625
19  -->  44373.71875
19  -->  44364.72265625
19  -->  44357.76171875
19  -->  44350.00390625
19  -->  44344.5078125
19  -->  44338.00390625
19  -->  44331.11328125
19  -->  44319.35546875
19  -->  44313.58203125
19  -->  44306.91015625
19  -->  44296.6875
19  -->  44294.1796875
20  -->  44277.765625
20  -->  44273.71875
20  -->  44267.9296875
20  -->  44260.7578125
20  -->  44252.90625
20  -->  44241.0703125
20  -->  44236.16015625
20  -->  44229.41796875
20  -->  44222.93359375
20  -->  44214.2734375
20  -->  44208.390625
20  -->  44203.12890625
20  -->  44200.5859375
20  -->  44193.65625
20  -->  44190.88671875
20  -->  44182.58203125
20  -->  441

36  -->  42393.03515625
36  -->  42385.734375
36  -->  42384.3125
36  -->  42376.60546875
36  -->  42373.91015625
36  -->  42369.6484375
36  -->  42363.12890625
37  -->  42353.25
37  -->  42351.109375
37  -->  42340.7578125
37  -->  42338.2421875
37  -->  42333.97265625
37  -->  42327.5859375
37  -->  42321.26171875
37  -->  42316.2734375
37  -->  42312.12890625
37  -->  42308.1953125
37  -->  42303.609375
37  -->  42295.3828125
37  -->  42292.22265625
37  -->  42286.515625
37  -->  42280.328125
37  -->  42277.171875
37  -->  42268.11328125
37  -->  42265.25390625
37  -->  42261.203125
37  -->  42257.609375
38  -->  42252.4375
38  -->  42246.28125
38  -->  42243.609375
38  -->  42236.13671875
38  -->  42242.765625
38  -->  42226.49609375
38  -->  42223.91796875
38  -->  42218.78125
38  -->  42214.50390625
38  -->  42208.83203125
38  -->  42204.375
38  -->  42200.80859375
38  -->  42197.04296875
38  -->  42193.14453125
38  -->  42186.6328125
38  -->  42182.16796875
38  -->  42175.59375


54  -->  41093.6328125
54  -->  41090.89453125
54  -->  41086.1796875
54  -->  41082.4296875
54  -->  41079.05078125
54  -->  41076.85546875
54  -->  41074.78125
55  -->  41068.109375
55  -->  41067.3203125
55  -->  41064.26171875
55  -->  41063.44921875
55  -->  41061.671875
55  -->  41058.8671875
55  -->  41058.1796875
55  -->  41053.97265625
55  -->  41051.8125
55  -->  41049.01953125
55  -->  41046.75
55  -->  41043.61328125
55  -->  41041.7421875
55  -->  41039.84375
55  -->  41036.578125
55  -->  41035.3125
55  -->  41030.296875
55  -->  41028.3515625
55  -->  41026.03125
55  -->  41022.67578125
56  -->  41018.5546875
56  -->  41015.4375
56  -->  41012.1015625
56  -->  41009.203125
56  -->  41006.8828125
56  -->  41004.2890625
56  -->  41002.09375
56  -->  40999.2109375
56  -->  40996.90234375
56  -->  40995.046875
56  -->  40999.1796875
56  -->  40993.51953125
56  -->  40992.9140625
56  -->  40990.36328125
56  -->  40989.375
56  -->  40986.36328125
56  -->  40984.4921875
56  -->

72  -->  40291.16796875
72  -->  40287.484375
72  -->  40284.6328125
72  -->  40283.3203125
72  -->  40282.0078125
72  -->  40280.6640625
72  -->  40278.14453125
72  -->  40277.61328125
73  -->  40275.36328125
73  -->  40274.47265625
73  -->  40272.3984375
73  -->  40270.69921875
73  -->  40268.5078125
73  -->  40266.1875
73  -->  40264.9921875
73  -->  40263.49609375
73  -->  40260.87890625
73  -->  40259.97265625
73  -->  40258.546875
73  -->  40257.0078125
73  -->  40253.6953125
73  -->  40252.015625
73  -->  40249.27734375
73  -->  40247.5625
73  -->  40245.99609375
73  -->  40242.2421875
73  -->  40238.703125
73  -->  40236.3828125
74  -->  40233.36328125
74  -->  40232.234375
74  -->  40229.73828125
74  -->  40225.62890625
74  -->  40221.96875
74  -->  40219.5703125
74  -->  40217.7734375
74  -->  40216.4453125
74  -->  40211.83984375
74  -->  40209.6484375
74  -->  40206.8984375
74  -->  40203.375
74  -->  40211.25
74  -->  40200.8671875
74  -->  40200.1953125
74  -->  40198.144

...

420  -->  34138.44921875
420  -->  34134.5390625
420  -->  34134.453125
420  -->  34134.0859375
420  -->  34134.4296875
420  -->  34133.609375
420  -->  34133.40234375
420  -->  34133.046875
421  -->  34132.82421875
421  -->  34132.921875
421  -->  34132.0859375
421  -->  34131.89453125
421  -->  34131.546875
421  -->  34131.20703125
421  -->  34130.7578125
421  -->  34130.25
421  -->  34129.93359375
421  -->  34129.359375
421  -->  34128.8828125
421  -->  34128.5
421  -->  34128.015625
421  -->  34127.60546875
421  -->  34127.23046875
421  -->  34126.7109375
421  -->  34126.546875
421  -->  34125.96875
421  -->  34125.73828125
421  -->  34125.4375
422  -->  34124.96875
422  -->  34126.90625
422  -->  34124.3828125
422  -->  34124.21484375
422  -->  34123.84375
422  -->  34123.4921875
422  -->  34123.39453125
422  -->  34122.703125
422  -->  34122.5234375
422  -->  34122.3515625
422  -->  34122.15234375
422  -->  34121.83984375
422  -->  34121.53515625
422  -->  34121.3671875
422  --> 

438  -->  34025.984375
438  -->  34025.8046875
438  -->  34025.67578125
438  -->  34025.3984375
438  -->  34025.11328125
438  -->  34024.5859375
438  -->  34024.83984375
438  -->  34023.96875
438  -->  34023.796875
438  -->  34023.5234375
438  -->  34023.2578125
438  -->  34023.09375
438  -->  34022.61328125
439  -->  34022.40234375
439  -->  34022.12890625
439  -->  34021.89453125
439  -->  34021.5625
439  -->  34021.2421875
439  -->  34021.06640625
439  -->  34020.58203125
439  -->  34027.5546875
439  -->  34020.0546875
439  -->  34019.93359375
439  -->  34019.41796875
439  -->  34019.03515625
439  -->  34018.859375
439  -->  34018.3125
439  -->  34018.171875
439  -->  34017.9140625
439  -->  34017.5234375
439  -->  34016.9140625
439  -->  34017.140625
439  -->  34016.3046875
440  -->  34016.14453125
440  -->  34015.71484375
440  -->  34015.4453125
440  -->  34015.0703125
440  -->  34014.6640625
440  -->  34014.3671875
440  -->  34013.75390625
440  -->  34013.4140625
440  -->  34012.

455  -->  33919.828125
455  -->  33919.46875
455  -->  33919.15625
456  -->  33918.94921875
456  -->  33918.6796875
456  -->  33918.1875
456  -->  33918.234375
456  -->  33917.703125
456  -->  33917.58984375
456  -->  33917.3671875
456  -->  33916.9453125
456  -->  33916.484375
456  -->  33916.01953125
456  -->  33915.80859375
456  -->  33915.4453125
456  -->  33915.0078125
456  -->  33915.26171875
456  -->  33914.4453125
456  -->  33914.3203125
456  -->  33914.0234375
456  -->  33913.6484375
456  -->  33913.2421875
456  -->  33912.890625
457  -->  33912.71484375
457  -->  33912.6171875
457  -->  33912.125
457  -->  33912.765625
457  -->  33911.578125
457  -->  33911.40625
457  -->  33911.07421875
457  -->  33910.6953125
457  -->  33910.53125
457  -->  33910.140625
457  -->  33909.9140625
457  -->  33909.8046875
457  -->  33909.578125
457  -->  33909.359375
457  -->  33909.1015625
457  -->  33908.6484375
457  -->  33908.32421875
457  -->  33908.0703125
457  -->  33907.68359375
457  -->

473  -->  33822.8359375
473  -->  33822.7265625
473  -->  33822.3203125
473  -->  33821.9296875
473  -->  33821.53125
473  -->  33821.26953125
473  -->  33821.0546875
473  -->  33820.8046875
473  -->  33820.578125
473  -->  33820.09375
473  -->  33820.3359375
473  -->  33819.42578125
473  -->  33819.22265625
473  -->  33818.796875
473  -->  33818.5
473  -->  33818.11328125
473  -->  33817.875
473  -->  33817.4609375
474  -->  33817.21484375
474  -->  33816.953125
474  -->  33816.46875
474  -->  33816.51953125
474  -->  33815.51171875
474  -->  33815.3125
474  -->  33814.984375
474  -->  33814.734375
474  -->  33814.36328125
474  -->  33814.0546875
474  -->  33813.52734375
474  -->  33813.21484375
474  -->  33812.84375
474  -->  33812.671875
474  -->  33812.48046875
474  -->  33812.37109375
474  -->  33812.09765625
474  -->  33811.78125
474  -->  33811.47265625
474  -->  33810.98828125
475  -->  33810.5546875
475  -->  33810.25
475  -->  33809.9765625
475  -->  33809.72265625
475  -->  

490  -->  33722.515625
490  -->  33722.2421875
490  -->  33721.7890625
490  -->  33721.4921875
490  -->  33721.19921875
490  -->  33720.90625
490  -->  33720.484375
490  -->  33720.32421875
490  -->  33720.078125
490  -->  33719.984375
490  -->  33719.6796875
490  -->  33719.3515625
491  -->  33719.1328125
491  -->  33718.7890625
491  -->  33718.671875
491  -->  33718.3359375
491  -->  33718.1171875
491  -->  33717.6875
491  -->  33717.48046875
491  -->  33717.1328125
491  -->  33716.9765625
491  -->  33716.78125
491  -->  33716.51171875
491  -->  33716.33203125
491  -->  33715.921875
491  -->  33715.5859375
491  -->  33715.28125
491  -->  33714.9375
491  -->  33714.70703125
491  -->  33714.37890625
491  -->  33714.1015625
491  -->  33713.74609375
492  -->  33713.46875
492  -->  33713.1796875
492  -->  33713.0859375
492  -->  33712.6484375
492  -->  33712.51953125
492  -->  33712.3125
492  -->  33711.90625
492  -->  33712.30859375
492  -->  33711.28515625
492  -->  33711.16015625
492  

In [4]:
a

tensor([[[ 2.8207, -0.6914,  0.1350,  ...,  0.3947,  0.1771,  1.0966],
         [ 1.0841,  0.1072,  0.9564,  ...,  0.8156,  1.4941,  0.2901],
         [ 1.0471, -2.8220,  0.7564,  ...,  1.5853,  0.3113,  0.9756],
         ...,
         [ 1.7909,  0.0200, -0.1498,  ...,  0.9267, -0.7358, -0.8116],
         [-0.3821,  0.4727,  0.9867,  ...,  0.5456, -2.2910, -0.5037],
         [-1.8541, -0.3556,  0.0814,  ..., -1.1367,  0.0377, -0.3836]],

        [[-0.5792, -0.6704, -0.4175,  ..., -1.7034, -0.6262,  0.7417],
         [-1.6853, -1.3461, -0.4814,  ..., -0.5070, -0.8683,  0.2545],
         [-0.9967, -0.1934, -0.5820,  ..., -0.4940,  0.0700, -0.7970],
         ...,
         [-0.5807,  0.3822,  0.2442,  ...,  0.5642, -1.0526,  1.1020],
         [ 0.8259, -1.6026,  0.4137,  ...,  0.5701,  0.7424, -0.0878],
         [-1.0734,  0.5916,  2.2781,  ...,  0.7902,  0.0676,  2.0488]],

        [[-0.0743,  0.7231,  0.3069,  ..., -0.4944, -0.0468,  1.1399],
         [-1.8574,  0.7944, -0.1564,  ..., -0